In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.metrics import Metric
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.ops import init_ops, math_ops
from tensorflow.python.keras.utils.generic_utils import to_list
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score  
import seaborn as sns
import codecs
import warnings 
warnings.filterwarnings("ignore") 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import coo_matrix, hstack, vstack

import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# from google.colab import files
# uploaded = files.upload()
# # # !mv 'train (1).csv'  train.csv
# import nltk
# nltk.download('stopwords')


In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.set_index("ID", inplace=True)


In [5]:
def data_text_preprocess(total_text, ind, col):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
#         total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            if not word in stop_words:
                string += word + " "
        
        df[col][ind] = string

In [6]:
for index, row in df.iterrows():
    if type(row['ABSTRACT']) is str:
        data_text_preprocess(row['ABSTRACT'], index, 'ABSTRACT')
    if type(row['TITLE']) is str:
        data_text_preprocess(row['TITLE'], index, 'TITLE')
df.head(10)

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,,
1,Reconstructing Subject Specific Effect Maps,Predictive models allow subject specific infer...,1,0,0,0,0,0
2,Rotation Invariance Neural Network,Rotation invariance translation invariance gre...,1,0,0,0,0,0
3,Spherical polyharmonics Poisson kernels polyha...,We introduce develop notion spherical polyharm...,0,0,1,0,0,0
4,A finite element approximation stochastic Maxw...,The stochastic Landau Lifshitz Gilbert LLG equ...,0,0,1,0,0,0
5,Comparative study Discrete Wavelet Transforms ...,Fourier transform infra red FTIR spectra sampl...,1,0,0,1,0,0
6,On maximizing fundamental frequency complement...,Let Omega subset mathbb R n bounded domain sat...,0,0,1,0,0,0
7,On rotation period shape hyperbolic asteroid 1...,We observed newly discovered hyperbolic minor ...,0,1,0,0,0,0
8,Adverse effects polymer coating heat transport...,The ability metallic nanoparticles supply heat...,0,1,0,0,0,0
9,SPH calculations Mars scale collisions role Eq...,We model large scale approx 2000km impacts Mar...,0,1,0,0,0,0


In [7]:

train, test = train_test_split(df, random_state=42, test_size=0.00001, shuffle=True)
# train.shape, test.shape
# test = pd.read_csv("cleaned_test_data.csv")

In [8]:

train_words_title, test_words_title = set(), set()
train_words_abstract, test_words_abstract = set(), set()
for index, row in train.iterrows():
    for  i in row['ABSTRACT'].split():
        train_words_abstract.add(i)
    for i in row['TITLE'].split():
        train_words_title.add(i)

for index, row in test.iterrows():
    for  i in row['ABSTRACT'].split():
        test_words_abstract.add(i)
    for i in row['TITLE'].split():
        test_words_title.add(i)
len(train_words_abstract - test_words_abstract), len(train_words_title - test_words_title)

(63471, 25180)

In [9]:
len(train_words_abstract), len(test_words_abstract), len(train_words_title), len(test_words_title)

(63555, 86, 25189, 9)

In [10]:
# w2i_title, w2i_abstract = {}, {}
# cnt = 0
# for i in train_words_abstract:
#     w2i_abstract.add()

In [11]:
# classifiers = {
#     "LogisiticRegression": LogisticRegression(),
#     "KNearest": KNeighborsClassifier(),
#     "Support Vector Classifier": SVC(),
#     "DecisionTreeClassifier": DecisionTreeClassifier(),
#     "MultinimialNB": MultinomialNB()
# }

In [12]:
abstract_max_len = 3000
title_max_len = 50
def get_label(row):
    label = [0, 0, 0, 0, 0, 0]
    columns = ['Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance']
    for col in range(len(columns)):
        if row[columns[col]] == 1:
            label[col] = 1
    return np.array(label)

def get_data(dframe):
#     X_title_vectors = count_vectorizer.transform(dframe['TITLE'])
#     X_abstract_vectors = count_vectorizer2.transform(dframe['ABSTRACT'])
#     X_title_vectors = hstack([X_title_vectors, X_abstract_vectors]).toarray()
    X_dframe = []
    y_dframe = []
    for index, row in tqdm(dframe.iterrows()):
        new_abstract_seq = []
        new_title_seq = []
        abstract = row["ABSTRACT"].split()
        title = row["TITLE"].split()
        for i in range(abstract_max_len):
            try: new_abstract_seq.append(abstract[i])
            except: new_abstract_seq.append("__PAD__")
        
        for i in range(title_max_len):
            try: new_title_seq.append(title[i])
            except: new_title_seq.append("__PAD__")
        y_dframe.append(get_label(row))        
        X_dframe.append([new_abstract_seq, new_title_seq])
    y_dframe = np.array(y_dframe)
    print((len(X_dframe), len(X_dframe[0])), y_dframe.shape)
    return X_dframe, y_dframe

In [13]:
X, y = get_data(train)
X_test, y_test = get_data(test)

20971it [00:19, 1058.13it/s]
1it [00:00, 493.51it/s]

(20971, 2) (20971, 6)
(1, 2) (1, 6)


In [14]:
batch_size = 32

In [15]:
sess = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session(sess)

In [16]:
def ElmoEmbedding1(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[abstract_max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]
def ElmoEmbedding2(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[title_max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [17]:
tf.compat.v1.disable_eager_execution()
elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
sess.run(tf.compat.v1.global_variables_initializer())
sess.run(tf.compat.v1.tables_initializer())

In [18]:
input_text1 = Input(shape=(title_max_len,), dtype="string", name="input")
embedding1 = Lambda(ElmoEmbedding2, output_shape=(100), name="embedding")(input_text1)
# input_text2 = Input(shape=(title_max_len,), dtype=tf.string)
# embedding2 = Lambda(ElmoEmbedding2, output_shape=(None, 1024))(input_text2)
# embedding = concatenate([embedding1, embedding2])
x = LSTM(128)(embedding1)
# # x_rnn = Bidirectional(LSTM(units=64, return_sequences=True
# #                            ), name="bi2")(x)

# x_rnn = Dense(256, activation="relu", name="dense1")(embedding1)
# x = add([x, x_rnn])  # residual connection to the first biLSTM
out = Dense(6, activation="sigmoid", name="dense")(x)

model = Model(inputs=input_text1, outputs=out)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
embedding (Lambda)           (32, None, 1024)          0         
_________________________________________________________________
lstm (LSTM)                  (32, 128)                 590336    
_________________________________________________________________
dense (Dense)                (32, 6)                   774       
Total params: 591,110
Trainable params: 591,110
Non-trainable params: 0
_________________________________________________________________


In [20]:
len(X), len(X[0][0]), len(X[0][1]), len(y), len(y[0])

(20971, 3000, 50, 20971, 6)

In [21]:
X1 = [X[i][1] for i in range(len(X))]
# total 655
X1 = X1[:655 * batch_size]
y1 = y[:655 * batch_size]
# X1_test = [X_test[i][1] for i in range(len(X_test))]
# X1_test = X1_test[:131 * batch_size]
# y1_test = y_test[:131 * batch_size]
history = model.fit(np.array(X1), y1, class_weight='balanced', batch_size=batch_size, epochs=4, verbose=1)

Train on 20960 samples
Epoch 1/4
20960/20960 [==============================] - 119s 6ms/sample - loss: 0.4214 - accuracy: 0.7990
Epoch 2/4
20960/20960 [==============================] - 120s 6ms/sample - loss: 0.3577 - accuracy: 0.8378
Epoch 3/4
20960/20960 [==============================] - 120s 6ms/sample - loss: 0.3138 - accuracy: 0.8641
Epoch 4/4
20960/20960 [==============================] - 120s 6ms/sample - loss: 0.2905 - accuracy: 0.8749


In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score
# total 655
X1_test = X1[-25 * batch_size:]
y1_test = y[-25 * batch_size:]
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
x_test = list(divide_chunks(X1_test, batch_size))

# y_test = divide_chunks(y_test, batch_size)
predictions = []
for x in tqdm(x_test):
  for i in model.predict(np.array(x)):
    predictions.append(i)


predictions = np.array(predictions)
thresholds=[.05, .04, .03, .02, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=predictions.copy()

    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y1_test, pred, average='micro')
    recall = recall_score(y1_test, pred, average='micro')
    f1 = f1_score(y1_test, pred, average='micro')
   
    print("Micro-average quality numbers for val " + str(val))
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

  0%|          | 0/25 [00:00<?, ?it/s]

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
100%|██████████| 25/25 [00:04<00:00,  5.65it/s]


Micro-average quality numbers for val 0.05
Precision: 0.2994, Recall: 0.8040, F1-measure: 0.4363
Micro-average quality numbers for val 0.04
Precision: 0.2929, Recall: 0.8677, F1-measure: 0.4380
Micro-average quality numbers for val 0.03
Precision: 0.2900, Recall: 0.9154, F1-measure: 0.4405
Micro-average quality numbers for val 0.02
Precision: 0.2695, Recall: 0.9473, F1-measure: 0.4197
Micro-average quality numbers for val 0.1
Precision: 0.3127, Recall: 0.6030, F1-measure: 0.4118
Micro-average quality numbers for val 0.2
Precision: 0.3264, Recall: 0.4806, F1-measure: 0.3887
Micro-average quality numbers for val 0.3
Precision: 0.3323, Recall: 0.4119, F1-measure: 0.3678
Micro-average quality numbers for val 0.4
Precision: 0.3357, Recall: 0.3264, F1-measure: 0.3310
Micro-average quality numbers for val 0.5
Precision: 0.3548, Recall: 0.2637, F1-measure: 0.3025
Micro-average quality numbers for val 0.6
Precision: 0.3537, Recall: 0.2249, F1-measure: 0.2749
Micro-average quality numbers for va

In [35]:
tdf.shape

(8989, 3)

In [36]:
def data_test_preprocess(total_text):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
#         total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            if not word in stop_words:
                string += word + " "
        
    return string
def get_test_data(dframe):
#     X_title_vectors = count_vectorizer.transform(dframe['TITLE'])
#     X_abstract_vectors = count_vectorizer2.transform(dframe['ABSTRACT'])
#     X_title_vectors = hstack([X_title_vectors, X_abstract_vectors]).toarray()
    X_dframe = []
    y_dframe = []
    for index, row in tqdm(dframe.iterrows()):
        new_abstract_seq = []
        new_title_seq = []
        abstract = row["ABSTRACT"].split()
        title = data_test_preprocess(row["TITLE"]).split()
        for i in range(abstract_max_len):
            try: new_abstract_seq.append(abstract[i])
            except: new_abstract_seq.append("__PAD__")
        
        for i in range(title_max_len):
            try: new_title_seq.append(title[i])
            except: new_title_seq.append("__PAD__")
        # y_dframe.append(get_label(row))        
        X_dframe.append(new_title_seq)
    # y_dframe = np.array(y_dframe)
    # print((len(X_dframe), len(X_dframe[0])), y_dframe.shape)
    return X_dframe

columns = ['Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance']
# for i in columns:
#   tdf[i] = pd.Series(np.zeros(len(tdf)))
X_test = get_test_data(tdf)



8989it [00:07, 1174.64it/s]


In [37]:
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
x_test = list(divide_chunks(X_test, batch_size))
for i in range(3): x_test[-1].append(['__PAD__'] * title_max_len)
# y_test = divide_chunks(y_test, batch_size)
predictions = []
for x in tqdm(x_test):
  for i in model.predict(np.array(x)):
    predictions.append([1 if a >= .4 else 0 for a in i])
len(predictions)

100%|██████████| 281/281 [00:45<00:00,  6.13it/s]


8992

In [38]:
for col in range(len(columns)):
    tdf[columns[col]] = pd.Series(np.array([i[col] for i in predictions[:-3]], dtype='int'))

In [39]:
np.array([i[0] for i in predictions], dtype='int').shape
tdf.tail()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
8984,29957,Supporting mixed-datatype matrix multiplicatio...,We approach the problem of implementing mixe...,1,0,0,0,0,0
8985,29958,An axiomatic basis for Blackwell optimality,In the theory of Markov decision processes (...,0,0,1,0,0,0
8986,29959,GeneVis - An interactive visualization tool fo...,GeneVis is a web-based tool to visualize com...,1,0,0,0,0,0
8987,29960,Quantifying the causal effect of speed cameras...,This paper quantifies the effect of speed ca...,1,0,0,0,0,0
8988,29961,Cube-magic labelings of grids,We show that the vertices and edges of a $d$...,0,1,0,0,0,0


In [40]:
predictions[2]

[1, 0, 0, 0, 0, 0]

In [41]:
model.predict(np.array(x_test[0]))[2]

array([0.72280216, 0.10740352, 0.08046275, 0.29803416, 0.01869419,
       0.01404533], dtype=float32)

In [42]:
model.predict(np.array(x_test[0]))

array([[0.6652789 , 0.12561578, 0.10697913, 0.29570392, 0.01836482,
        0.0139255 ],
       [0.03243079, 0.9366568 , 0.0191991 , 0.0198519 , 0.05218285,
        0.00171585],
       [0.72280216, 0.10740352, 0.08046275, 0.29803416, 0.01869419,
        0.01404533],
       [0.03180129, 0.93736804, 0.02956013, 0.01462927, 0.0284368 ,
        0.00193852],
       [0.7980398 , 0.04870107, 0.08097835, 0.37299606, 0.01735544,
        0.01391456],
       [0.4701133 , 0.05326371, 0.4039638 , 0.3998072 , 0.01700091,
        0.01606234],
       [0.20794904, 0.10298873, 0.4957281 , 0.24195234, 0.0521876 ,
        0.01755814],
       [0.848039  , 0.04924179, 0.04754103, 0.38169077, 0.02247345,
        0.01276019],
       [0.8195565 , 0.0667709 , 0.03500466, 0.43374592, 0.05039704,
        0.01039165],
       [0.08453792, 0.7403252 , 0.16051406, 0.03126684, 0.01515681,
        0.00543934],
       [0.14468615, 0.6288016 , 0.18627241, 0.04095264, 0.01252645,
        0.00744868],
       [0.86940265, 0

In [43]:
tdf.drop(['TITLE', 'ABSTRACT'], axis=1).to_csv('test_submit.csv', index=False)

In [44]:
from google.colab import files
files.download('test_submit.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>